In [ ]:
import re
from simplejson import loads
from snomedct_terminology_server.server.models import Description
from snomedct_terminology_server.server.utils import execute_query

In [ ]:
# word_stats_query = """
# SELECT json_object(array_agg(word::text),array_agg(nentry::text)) 
# FROM ts_stat('SELECT to_tsvector(''simple'', term) 
#     FROM denormalized_description_for_current_snapshot')"""
# word_stats_query = """SELECT * from description_terms"""
# results=execute_query(word_stats_query)
with open('description_term_counts.json') as f:
    results = loads(f.read())
expr_1 = '(\w+[-/]\w+|\w+)+'
expr_2 = '((\w+[-/])+\w+|\w+)+'
expr_3 = '((\w+[-/])+\w+|\w+[-/]\w+|\w+)+'
expr_4 = '((\w+[-/.])+\w+|\w+[-/]\w+|\w+|(\w/\w+)+|^/\w+|(^/\w+[-/])+\w+|(^/\w+[-/]\w+[-]\w+)+)+'
def words(text): return re.findall(expr_4, text.lower())

In [ ]:
WORDS = {}
for word, nentry in results.items():
    try:
        key = words(word)[0]
#         print({key[0]: word})
        WORDS.update({key[0]: int(nentry)})
    except IndexError:
        print("found no text match for: {}".format(word))
        
print(len(results.keys()))
print(len(WORDS.keys()))
print(len(set(results.keys()).difference(WORDS.keys())))
eliminated_results = set(results.keys()).difference(WORDS.keys())
print('/recombinant-b-subunit' in eliminated_results)
print({key: results[key] for key in eliminated_results})
print(WORDS['vapour-permeable'])

print(WORDS['recombinant-b-subunit'])

print(WORDS['frequency-doubled'])
# print(WORDS)

In [ ]:
# def words(text): return re.findall(r'\w+', text.lower())
# WORDS = Counter(words(text))

def P(word, N=sum(WORDS.values())):
    "Probability of `word`."
    return WORDS[word] / N

def correction(word):
    "Most probable spelling correction for word."
    return max(candidates(word), key=P)

def candidates(word):
    "Generate possible spelling corrections for word."
    return (known([word]) or known(edits1(word)) or known(edits2(word)) or [word])

def known(words):
    "The subset of `words` that appear in the dictionary of WORDS."
    return set(w for w in words if w in WORDS)

def edits1(word):
    "All edits that are one edit away from `word`."
    letters    = 'abcdefghijklmnopqrstuvwxyz'
    splits     = [(word[:i], word[i:])    for i in range(len(word) + 1)]
    deletes    = [L + R[1:]               for L, R in splits if R]
    transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R)>1]
    replaces   = [L + c + R[1:]           for L, R in splits if R for c in letters]
    inserts    = [L + c + R               for L, R in splits for c in letters]
    return set(deletes + transposes + replaces + inserts)

def edits2(word):
    "All edits that are two edits away from `word`."
    return (e2 for e1 in edits1(word) for e2 in edits1(e1))

In [ ]:
################ Test Code

def unit_tests():
    print(correction('peling'))
    assert correction('peling') == 'peeling'                  # insert
#     assert correction('korrectud') == 'corrected'           # replace 2
#     assert correction('bycycle') == 'bicycle'               # replace
#     assert correction('inconvient') == 'inconvenient'       # insert 2
#     assert correction('arrainged') == 'arranged'            # delete
    assert correction('vapour-preeable') == 'vapour-permeable'              # transpose + insert
#     assert correction('premeable') == 'permeable'             # transpose
    assert correction('clinicla') == 'clinical'               # transpose
    assert correction('cliniclaa') =='clinical'               # transpose + delete
    assert correction('wsa') ==  'was'
#     assert correction('word') == 'word'                     # known
#     assert correction('quintessential') == 'quintessential' # unknown
#     assert words('This is a TEST.') == ['this', 'is', 'a', 'test']
#     assert Counter(words('This is a test. 123; A TEST this is.')) == (
#            Counter({'123': 1, 'a': 2, 'is': 2, 'test': 2, 'this': 2}))
#     assert len(WORDS) == 32192
#     assert sum(WORDS.values()) == 1115504
#     assert WORDS.most_common(10) == [
#      ('the', 79808),
#      ('of', 40024),
#      ('and', 38311),
#      ('to', 28765),
#      ('in', 22020),
#      ('a', 21124),
#      ('that', 12512),
#      ('he', 12401),
#      ('was', 11410),
#      ('it', 10681)]
#     assert WORDS['the'] == 79808
#     assert P('quintessential') == 0
#     assert 0.07 < P('the') < 0.08
    return 'unit_tests pass'

def spelltest(tests, verbose=False):
    "Run correction(wrong) on all (right, wrong) pairs; report results."
    import time
    start = time.clock()
    good, unknown = 0, 0
    n = len(tests)
    for right, wrong in tests:
        w = correction(wrong)
        good += (w == right)
        if w != right:
            unknown += (right not in WORDS)
            if verbose:
                print('correction({}) => {} ({}); expected {} ({})'
                      .format(wrong, w, WORDS[w], right, WORDS[right]))
    dt = time.clock() - start
    print('{:.0%} of {} correct ({:.0%} unknown) at {:.0f} words per second '
          .format(good / n, n, unknown / n, n / dt))

def Testset(lines):
    "Parse 'right: wrong1 wrong2' lines into [('right', 'wrong1'), ('right', 'wrong2')] pairs."
    return [(right, wrong)
            for (right, wrongs) in (line.split(':') for line in lines)
            for wrong in wrongs.split()]

if __name__ == '__main__':
    print(unit_tests())
#     spelltest(Testset(open('spell-testset1.txt')))
#     spelltest(Testset(open('spell-testset2.txt')))
